In [25]:
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))  # makes it wider
import datetime
import math
import pandas as pd
import numpy as np
from os import listdir
from os.path import dirname
import plotly.express as px
import random
import itertools
from sklearn.metrics import confusion_matrix

### functions

In [26]:
def convert_str_to_date(row):
    if '-' in list(row):
        return datetime.datetime.strptime(str(row)[0:10] , '%Y-%m-%d')
    else:
        if len(row) in (9, 10):
            row = '0' + row if len(row) == 9 else row
            return datetime.datetime.strptime("-".join([row[6:], row[3:5], row[0:2]]), '%Y-%m-%d')
        else:
            return None

In [89]:
path = 'Sonuc/'
prediction = []
for f in listdir(dirname(path)):
    print(f.split("_")[1].split(".")[0])
    if '(' not in f.split("_")[1].split(".")[0]:
        _pred = pd.read_csv(path + f).rename(columns={'ds': 'Tarih', 'anomaly': 'ad_label_3', 'importance': 'anomaly_score_3'})
        _pred['BayiKodu'] = int(f.split("_")[1].split(".")[0])
    prediction += _pred.to_dict('results')
results = pd.DataFrame(prediction)
results['ad_label_2'] = results['ad_label_2'].apply(lambda x: 1 if x in [1, -1] else 0)
results.to_csv("data/result_volkan.csv")

7000186
1095039
4042154
7000179
4034571
4009155
7000347
7000435
4046118
64760
4048653
4037458(1)
63621
4035900
7000227
4041717
4044486
60843
501032
501026
1500809
4016317
7000025
7000018
63350
7000024
65053
1095158
62933
1060648
4044120
500661
4041104
7000408
1500203
4046119
7000420
500715
4042394
1095038
4045809
501621
7000193
1095010
1095012
7000191
501145
1500759
4044240
7000146
4015033
4040768
500703
501435
64978
501421
7000350
7000422
1500201
4046669
500677
64818
501555
4046286
4040807
4037331
65045
4044452
4042989
501794
1060474
7000033
7000027
4040806
1500823
500845
501583
7000219
64164
4040345
7000225
4039651
4041113
7000423
7000437
501346
501434
4046683
501408
4035679
4036370
1500764
60712
7000147
1500758
1095007
7000184
7000190
1095013
4037397(1)
501140
1095017
7000157
1500748
7000433
7000427
7000341
4037644
7000209
61742
501020
4040802
1500172
501791
60844
4040803
501579
7000426
1500211
4048654
501800
4046309
1095016
501157
1095014
500275
1500011
1500005
4042623
4019633
5018

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

### config

In [87]:
local_run = True
data_main_path = "data/" if local_run else "/tmp/mnt/data/"
prediction_main_path = "prediction/" if local_run else "/tmp/mnt/prediction/"
model_main_path = "models/" if local_run else "/tmp/mnt/models/"
data_read_paths = {'raw': 'ml3yil.csv',
                   'franchise_info': 'BayiIlkVeriGoderimTarihi.csv',
                   'model_data': 'data_for_lstm.csv',
                   'merged_results': "results" + "".join(str(datetime.datetime.now())[0:10].split("-")) + ".csv",
                   'results': "results.csv",
                   "external": "result_selim.csv",
                   "external_2": "result_volkan.csv",
                   "actual_values": "actual_results.csv",
                   "prediction_data": "prediction_data.xlsx"}
date_col = 'Tarih'
feature = 'KisiSayimi'

### data access

In [90]:
def get_data(path):
    if path.split("/")[-1] == data_read_paths['raw']:
        data = pd.read_csv(path, error_bad_lines=False, encoding="ISO-8859-1", sep=';')
        data[date_col] = data[date_col].apply(lambda x: convert_str_to_date(x))
        data['isoweekday'] = data[date_col].apply(lambda x: x.isoweekday())
        data['week_parts'] = data['isoweekday'].apply(lambda x: 1 if x in [1, 2, 3, 4, 5] else 0)
        franchise_Iot_starts = pd.read_csv(data_main_path + data_read_paths['franchise_info'], sep=';')
        franchise_Iot_starts = franchise_Iot_starts.rename(columns={'ilkverigonderimtarihi': 'd_r_start_date'})
        franchise_Iot_starts['d_r_start_date'] = franchise_Iot_starts['d_r_start_date'].apply(lambda x:
                                                                                              convert_str_to_date(
                                                                                                  x) if x == x else min(
                                                                                                  data['Tarih']))
        data = pd.merge(data, franchise_Iot_starts[['BayiKodu', 'd_r_start_date']], on='BayiKodu', how='left')
        data = data.query("d_r_start_date <= Tarih")
        return data, franchise_Iot_starts
    if path.split("/")[-1] == data_read_paths['prediction_data']:
        data = pd.read_excel(path)
        data['isoweekday'] = data[date_col].apply(lambda x: x.isoweekday())
        data['week_parts'] = data['isoweekday'].apply(lambda x: 1 if x in [1, 2, 3, 4, 5] else 0)
        return data
    if path.split("/")[-1] not in [data_read_paths['raw'], data_read_paths['prediction_data']]:
        data = pd.read_csv(path)
        if date_col not in data.columns:
            data = pd.read_csv(path, sep=';')
        data[date_col] = data[date_col].apply(lambda x: convert_str_to_date(x))
        return data

In [177]:
data_external = get_data(data_main_path + data_read_paths['external'])
data_external_2 = get_data(data_main_path + data_read_paths['external_2'])
prediction = get_data(data_main_path + data_read_paths['results'])
raw_data = get_data(data_main_path+data_read_paths['raw'])[0]

# external 1
prediction = pd.merge(prediction,
                         data_external[['BayiKodu', 'Tarih', 'anomaly', 'ops']].rename(
                             columns={'anomaly': 'ad_label_2', 'ops': 'anomaly_score_2'}),
                         on=['BayiKodu', 'Tarih'], how='left')
prediction['ad_label_2'] = prediction['ad_label_2'].apply(lambda x: 1 if x == -1 else 0)
prediction['ad_label_2'] = prediction['ad_label_2'].fillna(0)
prediction['anomaly_score_2'] = prediction['anomaly_score_2'].fillna(0)


predicted_franchises = list(prediction['BayiKodu'].unique())
data = pd.merge(raw_data, prediction[['BayiKodu', date_col, 'predict', 'anomaly_score',
                                  'ad_label', 'ad_label_2', 'anomaly_score_2']],
                on=['BayiKodu', date_col], how='left').sort_values(by='Tarih', ascending=True)

# external 2
data = pd.merge(data, data_external_2[['Tarih', 'BayiKodu', 'ad_label_3', 'anomaly_score_3']], 
                on=['Tarih', 'BayiKodu'], how='left')

data = data.fillna(0)
data = data.query("BayiKodu in @predicted_franchises")


for label in ['ad_label', 'ad_label_2', 'ad_label_3']:
    data[label] = data.apply(lambda row: row[label]
                                                            if row['Tarih'] > 
                                  datetime.datetime.strptime('2020-01-05', '%Y-%m-%d') else 0, axis=1)
    data[label] = data.apply(lambda row: 1 if row['KisiSayimi'] < 50 or
                                                               row['KisiSayimi'] > 2000 else row[label], axis=1)

    

In [178]:
results = data.query("Tarih > '2020-03-01'"
                    )[['anomaly_score', 'ad_label', 'predict', 'BayiKodu', 'Tarih', 
                       'ad_label_2', 'anomaly_score_2', 'ad_label_3', 'anomaly_score_3', 'KisiSayimi']]
results

,anomaly_score,ad_label,predict,BayiKodu,Tarih,ad_label_2,anomaly_score_2,ad_label_3,anomaly_score_3,KisiSayimi
736768,0.625455,0.0,361.621307,1500101,2020-03-02,0.0,0.000000,0.0,0.000000,473.0
736769,1.000000,1.0,424.696198,1500652,2020-03-02,1.0,0.588316,0.0,0.000000,88.0
736770,1.000000,1.0,335.913452,1500663,2020-03-02,0.0,0.000000,1.0,0.005393,622.0
736771,0.207611,0.0,1061.486206,7000449,2020-03-02,0.0,0.000000,0.0,0.000000,1348.0
736772,0.449738,0.0,471.784302,1500241,2020-03-02,0.0,0.000000,0.0,0.000000,642.0
...,...,...,...,...,...,...,...,...,...,...
751296,0.000000,0.0,0.000000,1095064,2020-03-19,0.0,0.000000,0.0,0.000000,158.0
751297,0.000000,0.0,0.000000,4042989,2020-03-19,0.0,0.000000,0.0,0.000000,214.0
751298,0.000000,0.0,0.000000,501599,2020-03-19,0.0,0.000000,0.0,0.000000,278.0
751299,0.000000,0.0,0.000000,1500230,2020-03-19,0.0,0.000000,0.0,0.000000,277.0


In [176]:
#actual = pd.read_excel("data/ML1-12MartSonucu.xlsx")
#results = pd.merge(actual.drop('KisiSayimi', axis=1), results, on=['BayiKodu', 'Tarih'], how='left')

In [179]:
results.pivot_table(index='BayiKodu', aggfunc={'ad_label': 'sum', 
                                               'ad_label_2': 'sum',
                                              'ad_label_3': 'sum'}
                   ).reset_index().sort_values(by='ad_label_3', ascending=False)

,BayiKodu,ad_label,ad_label_2,ad_label_3
624,7000302,18.0,18.0,18.0
335,1500253,18.0,18.0,18.0
445,4034560,18.0,18.0,18.0
390,1500743,18.0,18.0,18.0
313,1500110,18.0,18.0,18.0
...,...,...,...,...
349,1500332,1.0,0.0,0.0
348,1500329,0.0,0.0,0.0
345,1500317,0.0,0.0,0.0
343,1500306,1.0,0.0,0.0


In [180]:
print("number of data removed from train set :")
print(len(raw_data.query("KisiSayimi > 2000 or KisiSayimi < 50 or KisiSayimi != KisiSayimi")))
print("ratio :")
print(len(raw_data.query("KisiSayimi > 2000 or KisiSayimi < 50 or KisiSayimi != KisiSayimi")) / len(raw_data))

number of data removed from train set :
142397
ratio :
0.18953388854799874


In [181]:
print("number of total observing data :")
print(len(results))
print("number of franchises : ")
print(len(results['BayiKodu'].unique()))
print("number of days which has null or 0 values :")
len(results[results['KisiSayimi'] == 0])
print("number of data point KisiSayimi > 2k or < 50 :")
print(len(results.query("KisiSayimi > 2000 or KisiSayimi < 50")))
print("toatl label 1 anomaly result count ")
print(len(results.query("ad_label == 1 and KisiSayimi < 2000 and KisiSayimi > 50")))
print("toatl label 2 anomaly result count ")
print(len(results.query("ad_label_2 == 1 and KisiSayimi < 2000 and KisiSayimi > 50")))
print("toatl label 3 anomaly result count ")
print(len(results.query("ad_label_3 == 1 and KisiSayimi < 2000 and KisiSayimi > 50")))

number of total observing data :
12006
number of franchises : 
667
number of days which has null or 0 values :
number of data point KisiSayimi > 2k or < 50 :
1626
toatl label 1 anomaly result count 
265
toatl label 2 anomaly result count 
124
toatl label 3 anomaly result count 
224


In [182]:
import plotly.express as px
fig = px.histogram(results.query("ad_label == 1 and KisiSayimi < 2000 and KisiSayimi > 50"), 
                   x="anomaly_score", nbins=500)
fig.show()

In [246]:
results.query("actual == actual").to_csv("results" + "".join(str(datetime.datetime.now())[0:10].split("-")) + ".csv", index=False)

In [27]:
f.split(".")[0] + '_lstm.json'

'models/'

In [31]:
for f in listdir(dirname(model_main_path)):
    os.rename(model_main_path + f, f.split(".")[0] + '_lstm.json')

In [30]:
import os
os.rename(model_main_path + '4040499_1.json', model_main_path + '4040499_1_lstm.json')

In [294]:
batches = 25
if 20 < batches <= 30:
    print(batches)

25


In [297]:
min(raw_data['Tarih'])

Timestamp('2016-06-01 00:00:00')

In [306]:
import yaml
with open("configs.yaml") as file:
    config = yaml.full_load(file)

In [316]:
config['data_read_paths']

KeyError: 'data_read_paths'

In [317]:
config

{'db_connection': {'host': None,
  'port': None,
  'db': None,
  'user': None,
  'password': None},
 'csv_paths': {'data_read_paths': {'raw': 'ml3yil.csv',
   'franchise_info': 'BayiIlkVeriGoderimTarihi.csv',
   'model_data': 'data_for_lstm.csv',
   'results': 'results.csv',
   'external': 'result_selim.csv',
   'external_2': 'result_volkan.csv',
   'actual_values': 'actual_results.csv'},
  'data_main_path': {'local': 'data/', 'server': '/tmp/mnt/data/'},
  'model_main_path': {'local': 'data/', 'server': '/tmp/mnt/data/'}},
 'min_weeks': 4,
 'accepted_weeks_after_store_open': 5,
 'accepted_null_ratio': 0.4,
 'is_from_db': True,
 'hyper_parameters': {'lstm': {0: {'tsteps': 2,
    'lahead': 4,
    'lag': 2,
    'batch_size': 16,
    'batch_count': 16,
    'units': 4,
    'l1': 0.01,
    'l2': 0.01,
    'epochs': 5,
    'lr': 0.05,
    'activation': 'relu',
    'split_ratio': 0.8},
   1: {'tsteps': 3,
    'lahead': 10,
    'lag': 2,
    'batch_size': 32,
    'batch_count': 32,
    'units'

In [319]:
my_string = '2019-10-31'

# Create date object in given time format yyyy-mm-dd
my_date = datetime.strptime(my_string, "%Y-%m-%d")

print(my_date)
print('Type: ',type(my_date))

2019-10-31 00:00:00
Type:  <class 'datetime.datetime'>


In [327]:
type("0.1")(2.0)

'2.0'

In [322]:
a

'-'